<a href="https://colab.research.google.com/github/fzanart/Socialz/blob/wip/mu_lambda_ES.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install scipy --upgrade

In [100]:
import random
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import qmc

In [204]:
data = {}


def generate_random(number):
  values = []
  for i in range(10):
      values.extend([random.uniform(0, 1)])
  return values

#data['Betweeness'] = generate_random(10)
data['x'] = generate_random(10)
data['y'] = generate_random(10)

df = pd.DataFrame(data)
df

,x,y
0,0.751139,0.108134
1,0.698555,0.905625
2,0.996671,0.087320
3,0.037286,0.652930
4,0.376262,0.215034
5,0.440232,0.611504
6,0.499312,0.646958
7,0.819639,0.169245
8,0.620735,0.890788
9,0.385076,0.504895


mu: The number of parents selected each iteration.

lambda: Size of the population.

lambda / mu: Number of children generated from each selected parent.

In [209]:
sample = df.sample(frac=0.2, random_state=0)
sample

,x,y
2,0.234662,0.395465
8,0.901649,0.454467


In [210]:
replace = np.random.rand(2,2)
replace

array([[0.81336633, 0.41006907],
       [0.51782109, 0.86571386]])

In [211]:
df.loc[sample.index] = replace
df

,x,y
0,0.751139,0.108134
1,0.698555,0.905625
2,0.813366,0.410069
3,0.037286,0.652930
4,0.376262,0.215034
5,0.440232,0.611504
6,0.499312,0.646958
7,0.819639,0.169245
8,0.517821,0.865714
9,0.385076,0.504895


In [212]:
def objective(candidate):
  return qmc.discrepancy(candidate, method='L2-star',workers=-1)

In [213]:
objective(df)

0.09068656856060195

In [214]:
bounds = np.asarray([[0.0, 1.0], [0.0, 1.0]])

In [215]:
# check if a point is within the bounds of the search
def in_bounds(point, bounds):
	# enumerate all dimensions of the point
	for d in range(len(bounds)):
		# check if out of bounds for this dimension
		if point[d, 0] < bounds[d, 0] or point[d, 0] > bounds[d, 1]:
			return False
		elif point[d, 1] < bounds[d, 0] or point[d, 1] > bounds[d, 1]:
			return False
	return True

In [218]:
# evolution strategy (mu, lambda) algorithm
def es_comma(initial_record, objective, bounds, n_iter, step_size, mu, lam):
  best, best_eval = None, 1e+10
	# calculate the number of children per parent
  n_children = int(lam / mu)
	
  # initial population
  population = list()
  
  for i in range(lam):
    candidate = df.copy()
    sample = candidate.sample(frac=0.2, random_state=i).index
    candidate.loc[sample] = np.random.rand(2,2)
    population.append((sample, candidate))

  # perform the search
  for epoch in range(n_iter):
    # evaluate the fitness for the population
    scores = [objective(candidate[1]) for candidate in population]
    # rank scores in ascending order
    ranks = np.argsort(np.argsort(scores))
    # select the indexes for the top mu ranked solutions, drop the worse results	
    selected = [i for i,_ in enumerate(ranks) if ranks[i] < mu]
    # create offspring from parents
    offspring = list()
    for i in selected:
      # check if this parent is the best solution ever seen
      if scores[i] < best_eval:
        best, best_eval = population[i], scores[i]
        print('%d, Best eval: = %.5f' % (epoch, best_eval))
        # create offspring for parent
        for j in range(n_children):
          aux = None
          while aux is None or not in_bounds(aux.to_numpy(), bounds):
            aux = population[i][1].loc[population[i][0]] 
            aux = aux + np.random.rand(2,2) * step_size

          population[i][1].loc[population[i][0]] = aux
          child = (population[i][0], population[i][1])
          offspring.append(child)

      # replace population with children
      # population = offspring

    return [best, best_eval]

In [219]:
# define range for input
bounds = np.asarray([[0.0, 1.0], [0.0, 1.0]])
# define the total iterations
n_iter = 5000
# define the maximum step size
step_size = 0.15
# number of parents selected
mu = 20
# the number of children generated by parents
lam = 100
# perform the evolution strategy (mu, lambda) search
best, score = es_comma(df, objective, bounds, n_iter, step_size, mu, lam)

0, Best eval: = 0.08377
0, Best eval: = 0.07527
0, Best eval: = 0.07290
0, Best eval: = 0.06713
